## A Prime number generator


See https://www.youtube.com/watch?v=_gCKX6VMvmU

Let's write a Julia function that implements the prime number generator described in the Numberphile video. It's a good candidate for a memoizing function. Here it is:

In [1]:
using Memoize

In [2]:
@memoize function mygenprime(n::Integer)
    if n == 1  
        2.920050977316
    else
        f = mygenprime(n-1)
        floor(f)*(f - floor(f) + 1)
    end
end            

mygenprime (generic function with 1 method)

In [3]:
[(k, floor(Int, mygenprime(k))) for k = 1 : 15]

15-element Array{Tuple{Int64,Int64},1}:
 (1, 2)
 (2, 3)
 (3, 5)
 (4, 7)
 (5, 11)
 (6, 13)
 (7, 17)
 (8, 19)
 (9, 23)
 (10, 29)
 (11, 31)
 (12, 37)
 (13, 40)
 (14, 42)
 (15, 43)

Oops! Forty isn't a prime. Neither is forty-two. What's the story? It's crazy to think that _all_ primes can be encoded into the number $2.920050977316$. Almost surely, to generate the primes past $37$, we need a more digits of the constant $2.920050977316$.  

Fortunately, the video tells us how to compute this constant. Not surprisingly, the constant is expressed as an _infinite series_. The best we can do is sum its first few terms.

Here is a function `Q` that sums the first `n` terms of the infinite series for this constant. So that we don't have to worry about rounding errors, we'll sum this using exact rational numbers.  The optional package `Primes` has a function that generates the primes for us.

In [4]:
using Primes

In [5]:
function Q(n::Integer)
    prod = BigInt(1)
    s = BigInt(0)
    for k = 1 : n
        pk = prime(k)
        s += (pk-1)//prod
        prod *= pk
    end
    s
end

Q (generic function with 1 method)

Summing the first $64$ terms, we have

In [6]:
Q(64)

2982076387136897711868778837562533128923101076886460857776351024737697552705029412066629838751794837984142947810782961//1021241207877052728809092149097626662644537663247243355111401451312772219284716228359734476103267543069998638899157773

Converting to a decimal, we have

In [7]:
BigFloat(Q(64))

2.92005097731613471209256291711201946800272789932142671977268253310773377212777

In [8]:
Q(128)

5227554455733777871260305151323808032492772521481472963485838824360748775617077180526329978266811309524859073074134253177854209904470343077182696221368015025622622213062490474563836827606091357779777088425776324911260290461579448439440774443274564890853890530370099364635758182172299843563765//1790227121493100199266413788084742281796363638940326181979063572262335989939146297006758045608216781053380619178432229671020652188884002829412501970104427630569751221728626885262586272945339984183616494370041526619082760890373018744558545629914917132431516486188023601797767156032558566683497

In [9]:
BigFloat(Q(128))

2.92005097731613471209256291711201946800272789932142671977268253310773377212777

The difference $Q(128) - Q(64)$ is small--and I mean really small.

In [10]:
BigFloat(Q(128)-Q(64))

1.60409249577861863914907299171977958705313392093787080488841313278988811892698e-123

Let's use the value of $Q(128)$ to generate primes. Let's rewrite our function and make the value of its first term an argument.

In [11]:
@memoize function mygenprime2(n::Integer, f0::Rational)
    if n == 1  
       f0
    else
       f = mygenprime2(n-1,f0)
       floor(f)*(f - floor(f) + 1)
    end
end            

mygenprime2 (generic function with 1 method)

In [12]:
f0 = Q(128);

In [13]:
[(k, prime(k), floor(Int, mygenprime2(k,f0))) for k = 1 : 128]

128-element Array{Tuple{Int64,Int64,Int64},1}:
 (1, 2, 2)
 (2, 3, 3)
 (3, 5, 5)
 (4, 7, 7)
 (5, 11, 11)
 (6, 13, 13)
 (7, 17, 17)
 (8, 19, 19)
 (9, 23, 23)
 (10, 29, 29)
 (11, 31, 31)
 (12, 37, 37)
 (13, 41, 41)
 ⋮
 (117, 643, 643)
 (118, 647, 647)
 (119, 653, 653)
 (120, 659, 659)
 (121, 661, 661)
 (122, 673, 673)
 (123, 677, 677)
 (124, 683, 683)
 (125, 691, 691)
 (126, 701, 701)
 (127, 709, 709)
 (128, 719, 718)

As a guess, in general using $Q(n)$ as the first term, we can correctly find the first $n-1$ primes, but not the n-th prime; example:

In [14]:
(prime(263), floor(Int, mygenprime2(263, Q(264))))

(1669, 1669)

In [15]:
(prime(264), floor(Int, mygenprime2(264, Q(264))))

(1693, 1692)

__Question__: Could we use the same to scheme to generate terms of other sequences? Say we have an integer-valued sequence $p_1, p_2, p_3, \dots $.  We calculate the constant the same way, but instead of the primes, we use the terms of some other sequence; thus

In [16]:
function QQ(F::Function, n::Integer)
    prod = BigInt(1)
    s = BigInt(0)
    for k = 1 : n
        pk = F(k)
        s += (pk-1)//prod
        prod *= pk
    end
    s
end

QQ (generic function with 1 method)

Let's try everyone's favorite sequence:

In [17]:
@memoize function fib(n::Integer)
    if n == 1 || n == 2
        1
    else
        fib(n-2) + fib(n-1)
    end
end

fib (generic function with 1 method)

Ha! We get the terms of the Fibonacci sequence, but the sequence is _shifted_ by two terms. Actually, since the constant is
$$
   (p_1 -1) + \frac{p_2 -1}{p_1}  + \cdots,
$$
the first two terms of the series both vanish, so the values of $p_1$ and $p_2$ don't get incorporated into the constant.

In [18]:
f0 = QQ(fib, 128);

In [19]:
[(k, fib(k), floor(BigInt, mygenprime2(k,f0))) for k = 1 : 24]

24-element Array{Tuple{Int64,Int64,BigInt},1}:
 (1, 1, 2)
 (2, 1, 3)
 (3, 2, 5)
 (4, 3, 8)
 (5, 5, 13)
 (6, 8, 21)
 (7, 13, 34)
 (8, 21, 55)
 (9, 34, 89)
 (10, 55, 144)
 (11, 89, 233)
 (12, 144, 377)
 (13, 233, 610)
 (14, 377, 987)
 (15, 610, 1597)
 (16, 987, 2584)
 (17, 1597, 4181)
 (18, 2584, 6765)
 (19, 4181, 10946)
 (20, 6765, 17711)
 (21, 10946, 28657)
 (22, 17711, 46368)
 (23, 28657, 75025)
 (24, 46368, 121393)

With the conjecture that maybe we want to exclude $1$ from the range of the sequence, let's try $k \mapsto k+1$ 

In [20]:
xxx = QQ(x -> x+1, 64)

70911686272574852599660141113652666333760118218640746369131464891915518607035687428261//26086951518479474527844035636340764967694390660885113527714205258062233600000000000000

OK--this seems to work OK:

In [21]:
[(k+1, floor(BigInt, mygenprime2(k, xxx))) for k = 1 : 24]

24-element Array{Tuple{Int64,BigInt},1}:
 (2, 2)
 (3, 3)
 (4, 4)
 (5, 5)
 (6, 6)
 (7, 7)
 (8, 8)
 (9, 9)
 (10, 10)
 (11, 11)
 (12, 12)
 (13, 13)
 (14, 14)
 (15, 15)
 (16, 16)
 (17, 17)
 (18, 18)
 (19, 19)
 (20, 20)
 (21, 21)
 (22, 22)
 (23, 23)
 (24, 24)
 (25, 25)

What about a sequence of squares?  We'll try $2^2, 3^3, \dots$. First, we compute the constant

In [22]:
xxx = QQ(x -> (1+x)^2, 128);

This fails from the start--maybe there is an upper bound on the growth of the sequence that allows the method to work. And maybe there is some condition on the difference between consecutive terms as well.

In [23]:
[(k, (1+k)^2, floor(BigInt, mygenprime2(k, xxx))) for k = 1 : 16]

16-element Array{Tuple{Int64,Int64,BigInt},1}:
 (1, 4, 5)
 (2, 9, 7)
 (3, 16, 9)
 (4, 25, 10)
 (5, 36, 11)
 (6, 49, 18)
 (7, 64, 31)
 (8, 81, 54)
 (9, 100, 93)
 (10, 121, 141)
 (11, 144, 147)
 (12, 169, 277)
 (13, 196, 356)
 (14, 225, 484)
 (15, 256, 758)
 (16, 289, 1512)

Searching `oeis` (https://oeis.org/) for the sequence $5,7,9,10,11,18,31, \dots $ doesn't find a match.  You might like to try sequences with other growth rates.